In [ ]:
# If PEPit is not installed yet, you can run this cell.
!pip install pepit

# Code description

In this notebook, we numerically verify Lemma C.4 from 

> "Convergence of Proximal Point and Extragradient-Based Methods Beyond Monotonicity: the Case of Negative Comonotonicity"

## Problem

We consider the following problem
       $$ \text{find } x^\ast \text{ such that }F(x^\ast) = 0$$
where $F:\mathbb{R}^d \to \mathbb{R}^d$ is a maximal $\rho$-negatively comonotone and $L$-Lipschitz operator.

## Algorithm

We consider optimistic gradient method (OG) with stepsize policy $\gamma_1=\gamma_2=\gamma$ with $\tilde{x}^0 = x^0$ and for all $k > 0$
$$
\tilde{x}^k = x^k - \gamma F(\tilde{x}^{k-1}),
$$
$$
x^{k+1} = x^k - \gamma F(\tilde{x}^k)
$$

## The goal

Let $k \geq 1$, $\rho \leq \frac{5}{62L}$ and $4\rho \leq \gamma \leq \frac{10}{31L}$, and $\Psi_k = \|F(x^{k})\|^2 + \|F(x^{k}) - F(\tilde{x}^{k-1})\|^2$. Lemma C.8 states
$$
\Psi_{k+1} - \Psi_k \leq -\frac{1}{100}\|F(\tilde{x}^k) - F(\tilde{x}^{k-1})\|^2.
$$


Below we verify this inequality numerically

In [1]:
# Run this code before executing the cell below
from math import sqrt
import numpy as np
from PEPit import PEP

from PEPit.function import Function


class LipschitzNegativeComonotoneOperator(Function):
    
    def __init__(self,
                 rho,
                 L=1.,
                 is_leaf=True,
                 decomposition_dict=None,
                 reuse_gradient=True):
        
        super().__init__(is_leaf=is_leaf,
                         decomposition_dict=decomposition_dict,
                         reuse_gradient=True)
        # Store L and rho
        self.rho = rho
        self.L = L

        if self.L == np.inf:
            print("\033[96m(PEPit) The class of Lipschitz negative comonotone operators is necessarily continuous.\n"
                  "To instantiate an operator, please avoid using the class LipschitzNegativeComonotoneOperator with\n"
                  " L == np.inf. Instead, please use the class NegativeComonotoneOperator (which accounts for the fact\n"
                  "that the image of the operator at certain points might not be a singleton).\033[0m")

    def add_class_constraints(self):

        for point_i in self.list_of_points:

            xi, gi, fi = point_i

            for point_j in self.list_of_points:

                xj, gj, fj = point_j

                if (xi != xj) | (gi != gj):
                    # Interpolation conditions of negative comonotone operator class
                    self.add_constraint((gi - gj) * (xi - xj) + self.rho * (gi - gj)**2 >= 0)
                    # Interpolation conditions of Lipschitz operator class
                    self.add_constraint((gi - gj)**2 - self.L**2 * (xi - xj)**2 <= 0)

In [2]:
##########################################################
# parameters: MODIFY HERE!

# pick the parameters for which you want to verify
# the inequality (numerically)
L = 1;
gamma = 1.0 / (3.1*L);
rho = gamma / 4;

##########################################################
if rho > 5/62/L:
    print('The value of rho is not within the prescribed bounds')
if gamma < 4*rho:
    print('The values of gamma and rho are not within the prescribed bounds')
if gamma > 10/31/L:
    print('The value of gamma is not within the prescribed bounds')


# verbose & verification tolerance options
verbose = 0;
tolerance = 1e-5;

# (0) Initialize an empty PEP
problem = PEP()

# (1) Set up the problem class
L  =  L; rho = rho; # F is 1-Lipschitz and \rho-negative comonotone
F  = problem.declare_function(LipschitzNegativeComonotoneOperator, L=L, rho=rho)
xs = F.stationary_point();  # x^* is a solution

# (2) Set up the starting points
tx0 = problem.set_initial_point() # this is \tilde{x}^0
x1  = problem.set_initial_point() # this is x^1

# (3) Run the algorithm
tx1     = x1 - gamma * F.gradient(tx0); 
x2      = x1 - gamma * F.gradient(tx1);

# (3) define the expressions (recall that our objective is to verify that
#     psi2 - psi1 - residual<=0 for all F and sequence generated by the
#     optimistic gradient method.

psi1 = F.gradient(x1)**2 +  ( F.gradient(x1) - F.gradient(tx0) )**2;
psi2 = F.gradient(x2)**2 + ( F.gradient(x2) - F.gradient(tx1) )**2;
residual =  -0.01 * ( F.gradient(tx1) - F.gradient(tx0) )**2;

# (4) Set up the performance measure:
expression_to_verify = psi2 - psi1 - residual;
problem.set_performance_metric(expression_to_verify);

# (5) Solve the PEP
worstcase_value = problem.solve(verbose=verbose)


# (6) is the potential verified? Success if expression2 - expression1 <= 0 for the 
#     choice of the parameters above.
print('Did PEPit verify the potential (within prescribed numerical precision)? {:}  \t'.format(worstcase_value<tolerance))
worstcase_value # this should be close to zero

Did PEPit verify the potential (within prescribed numerical precision)? True  	


-1.7728707391029275e-10